# Simple implementation af Kalman filteret
\begin{align*}
    \intertext{kinematic and observations model}
    \mathbf{x}[n]&=\Phi \mathbf{x}[n-1] + \mathbf{u}[n], \\
    \mathbf{z}[n]&=\mathbf{x}[n] + \mathbf{w}[n],\\
    \intertext{the prediction and prediction mean square error,}
    \hat{\mathbf{x}}[n|n-1]&=\Phi\hat{\mathbf{x}}[n-1|n-1], \\
    M[n|n-1]&=\Phi M[n-1|n-1]\Phi^\top + S_{\mathbf{u}}, \\
    \intertext{the Kalman gain,}
    K[n]&=M[n|n-1](S_{\mathbf{w}}+M[n|n-1])^{-1},\\
    \intertext{and the correction and estimate mean square error,}
    \hat{\mathbf{x}}[n|n]&=\hat{\mathbf{x}}[n|n-1]+K[n](\mathbf{z}[n]-\hat{\mathbf{x}}[n|n-1]),\\
    M[n|n]&=(I-K[n])M[n|n-1].
\end{align*}

In [2]:
import numpy as np
import matplotlib.pyplot as plt


distance = np.load("sat_2_100deg/distance.npy")
velocity = np.load("sat_2_100deg/velocity.npy")

In [3]:
class Kalman:
    z = []

    x_predictions = []
    x_corrections = []

    M_predictions = []
    M_corrections = []

    K = []

    def __init__(self, phi, S_u, S_w, x_init_guess, M_init_guess, x):
        # Save filter constants and data
        self.phi = phi
        self.S_u = S_u
        self.S_w = S_w
        self.dim = x.shape
        self.x = x

        # Save initial predicitons and initial measurement
        self.x_predictions.append(x_init_guess)
        self.M_predictions.append(M_init_guess)
        self.z.append(x[0] + np.random.normal(np.zeros(self.dim), S_w, self.dim))


    def __make_guess(self):
        # Create guess of the next step
        x_guess = self.phi*self.x_predictions[-1]
        M_guess = self.phi*self.M_predictions[-1]*self.phi.T + self.S_u

        # Append the guess
        self.x_predictions.append(x_guess)
        self.M_predictions.append(M_guess)

        # create new observation
        n = len(self.x_predictions)
        obs = self.x[n-1] + np.random.normal(np.zeros(self.dim), self.S_w, self.dim)
        self.z.append(obs)


    def __kalman_gain(self):
        # Calculate and append the Kalman gain
        gain = self.M_predictions[-1]*np.linalg.inv(self.S_w+self.M_predictions[-1])
        self.K.append(gain)


    def __correction(self):
        # Calculate the correciton step of the process
        x_correct = self.x_predictions[-1]+self.K[-1]*(self.z[-1]-self.x_predictions[-1])
        M_correct = (np.eye(self.dim)-self.K[-1])*self.M_predictions[-1]

        # Append the corrections
        self.x_corrections.append(x_correct)
        self.M_corrections.append(M_correct)

    # Lav en offentlig funktion som løber funktionerne igennem og udregner kalman for det givne data